# 1DATOS: EXPLORACIÓN Y PREPARACIÓN DE DATOS

Este notebook demuestra el funcionamiento del módulo 1datos, que se encarga de cargar, explorar y preparar los datos de retornos diarios para la optimización de carteras.

## Paso 1: Importar librerías y módulo

In [ ]:
import sys
import os
import importlib
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importar módulo (los nombres que empiezan con números requieren importlib)
datos = importlib.import_module('1datos')

print("Librerías importadas correctamente")

## Paso 2: Cargar datos de retornos

In [ ]:
ruta_csv = '../data/prod_long_sharpe_u50_20260116_v5_train_dataset.csv'
retornos = datos.cargar_retornos(ruta_csv)

print(f"\nDimensiones del DataFrame: {retornos.shape}")
print(f"Primeras filas:")
print(retornos.head())
print(f"\nÚltimas filas:")
print(retornos.tail())

## Paso 3: Calcular estadísticas básicas por activo

In [ ]:
stats = datos.calcular_estadisticas_basicas(retornos)

print("Estadísticas básicas por activo:")
print(stats.head(10))

print("\nResumen estadístico:")
print(stats.describe())

## Paso 4: Visualizar estadísticas

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Sharpe histórico
axes[0, 0].barh(range(10), stats['sharpe_historico'].head(10).values)
axes[0, 0].set_yticks(range(10))
axes[0, 0].set_yticklabels(stats.head(10).index)
axes[0, 0].set_xlabel('Sharpe Ratio Histórico')
axes[0, 0].set_title('Top 10 Activos por Sharpe Ratio')
axes[0, 0].grid(True, alpha=0.3)

# Rentabilidad anual
axes[0, 1].hist(stats['media_anual'].values * 100, bins=20, edgecolor='black')
axes[0, 1].set_xlabel('Rentabilidad Anualizada (%)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución de Rentabilidades Anuales')
axes[0, 1].grid(True, alpha=0.3)

# Volatilidad anual
axes[1, 0].hist(stats['std_anual'].values * 100, bins=20, edgecolor='black', color='orange')
axes[1, 0].set_xlabel('Volatilidad Anualizada (%)')
axes[1, 0].set_ylabel('Frecuencia')
axes[1, 0].set_title('Distribución de Volatilidades Anuales')
axes[1, 0].grid(True, alpha=0.3)

# Riesgo vs Rentabilidad
axes[1, 1].scatter(stats['std_anual'].values * 100, stats['media_anual'].values * 100, alpha=0.6)
axes[1, 1].set_xlabel('Volatilidad Anualizada (%)')
axes[1, 1].set_ylabel('Rentabilidad Anualizada (%)')
axes[1, 1].set_title('Riesgo vs Rentabilidad')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('modulo1_estadisticas.png', dpi=300, bbox_inches='tight')
plt.show()

## Paso 5: Analizar correlaciones entre activos

In [ ]:
stats_corr = datos.analizar_correlaciones(retornos)

print("Estadísticas de correlación:")
print(f"  Correlación media: {stats_corr['media']:.4f}")
print(f"  Correlación mínima: {stats_corr['min']:.4f}")
print(f"  Correlación máxima: {stats_corr['max']:.4f}")
print(f"  Desviación estándar: {stats_corr['std']:.4f}")

## Paso 6: Visualizar matriz de correlaciones

In [ ]:
fig = datos.visualizar_correlaciones(stats_corr['matriz'], ruta_guardado='modulo1_correlaciones.png')
plt.show()

## Paso 7: Análisis temporal

In [ ]:
analisis_temp = datos.analizar_temporal(retornos)

print("Análisis temporal completado")
print(f"  Retornos acumulados: {analisis_temp['retornos_acumulados'].shape}")
print(f"  Índice de mercado: {analisis_temp['indice_mercado'].shape}")
print(f"  Volatilidad rolling: {analisis_temp['volatilidad_rolling'].shape}")

## Paso 8: Visualizar evolución temporal

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Retornos acumulados (primeros 10 activos)
ret_acum = analisis_temp['retornos_acumulados']
for col in ret_acum.columns[:10]:
    axes[0].plot(ret_acum.index, ret_acum[col].values, alpha=0.6, linewidth=1)
axes[0].plot(analisis_temp['indice_mercado'].index, 
             (1 + analisis_temp['indice_mercado']).cumprod().values, 
             'k-', linewidth=2, label='Índice Mercado')
axes[0].set_xlabel('Día')
axes[0].set_ylabel('Valor Acumulado')
axes[0].set_title('Evolución de Retornos Acumulados (10 primeros activos)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Volatilidad rolling promedio
vol_rolling = analisis_temp['volatilidad_rolling']
vol_promedio = vol_rolling.mean(axis=1)
axes[1].plot(vol_rolling.index, vol_promedio.values * 100, linewidth=2)
axes[1].set_xlabel('Día')
axes[1].set_ylabel('Volatilidad Anualizada (%)')
axes[1].set_title('Volatilidad Rolling Promedio (63 días)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('modulo1_temporal.png', dpi=300, bbox_inches='tight')
plt.show()

## Paso 9: Preparar datos para optimización que se llame data y

In [ ]:
preparador = datos.PreparadorDatos(retornos, rf_anual=0.02)
preparador.calcular_estadisticas()

mu, Sigma, rf = preparador.obtener_estadisticas()

print("Datos preparados para optimización:")
print(f"  Vector μ (rentabilidades esperadas): shape {mu.shape}")
print(f"  Matriz Σ (covarianza): shape {Sigma.shape}")
print(f"  Tasa libre de riesgo: {rf:.4f} ({rf*100:.2f}% anual)")
print(f"\n  Rentabilidad promedio: {mu.mean()*100:.2f}% anual")
print(f"  Volatilidad promedio (diagonal Σ): {np.sqrt(np.diag(Sigma)).mean()*100:.2f}% anual")

## Resumen del módulo 1datos

El módulo 1datos ha completado exitosamente:

1. Carga y validación de datos de retornos diarios
2. Cálculo de estadísticas básicas por activo
3. Análisis de correlaciones entre activos
4. Análisis temporal de retornos y volatilidad
5. Preparación de vectores μ y matriz Σ anualizados

Los datos están listos para ser usados en el módulo 2markowitz (Optimización de Markowitz).